<a href="https://colab.research.google.com/github/spencerklawans/466-Project/blob/first-model/WildfirePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wildfire Prediction
Brought to you by Gooogle Earth Engine

In [2]:
# Import libraries.
import ee
import folium

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
#Init KDD libraries

%matplotlib inline

import numpy as np
import pandas as pd
import sklearn

In [ ]:
California = ee.Geometry.Rectangle(-124.18, 41.39, -114.35, 32.75)

# Import our fire events as features
modis_fires = ee.FeatureCollection('JRC/GWIS/GlobFire/v2/FinalPerimeters').filterBounds(California)
# filter for only fires with greater than 10,000,000 square meters
modis_fires = modis_fires.filter(ee.Filter.gt('area', 10000000))

# Add a band for if this is a really big fire or not. 
# NOTE: We are using fire size as a proxy for severity
massive_fire_threshold = 100000000
massive_fires = modis_fires.filter(ee.Filter.gt('area', massive_fire_threshold)).map(lambda f: f.set({'massive?': 1}))  # 158 fires
other_fires = modis_fires.filter(ee.Filter.lte('area', massive_fire_threshold)).map(lambda f: f.set({'massive?': 0}))   # 609 fires
modis_fires = massive_fires.merge(other_fires)

In [12]:
slo = ee.Geometry.Point([-120.4358, 35.3102])

# Import ImageCollections
landfire_prs = ee.Image("LANDFIRE/Fire/PRS/v1_2_0/CONUS")
landfire_evc = ee.Image("LANDFIRE/Vegetation/EVC/v1_4_0/CONUS")

In [29]:
# Set visualization parameters. I'm pretty sure min and max should track to the min and max values of the set at https://developers.google.com/earth-engine/datasets/catalog/LANDFIRE_Fire_PRS_v1_2_0#bands
prs_viz_params = {
    'bands': ['PRS']
}

evc_vis_params = {
    'bands': ['EVC'],
    'opacity':.5
}
fires_viz_params = {
    'palette': ['#FF0000', '#0000FF'],
    'min': 0,
    'max': 1
}

# Create a folium map object.
my_map = folium.Map(location=[35.3102, -120.4358], zoom_start=8)

# Add the fire severity & existing vegetation type images to the map object.
my_map.add_ee_layer(landfire_prs, prs_viz_params, 'Percent Replacement-Severity Fire')
my_map.add_ee_layer(landfire_evc, evc_vis_params, 'Existing Vegetation Cover')
my_map.add_ee_layer(ee.Image().paint(modis_fires, 'massive?'), fires_viz_params, 'Fire Events')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

In [ ]:
# Training

training = landfire_evc.sampleRegions(landfire_evc)